# Monitoring unstructured data: Code Practice

In [ ]:
import pandas as pd
import numpy as np

from sklearn import datasets, ensemble, model_selection

from evidently import ColumnMapping
from evidently.report import Report

from evidently.metric_preset import TextOverviewPreset

from evidently.metrics import ColumnSummaryMetric
from evidently.metrics import ColumnCorrelationsMetric
from evidently.metrics import ColumnDistributionMetric
from evidently.metrics import ColumnDriftMetric
from evidently.metrics import ColumnValueRangeMetric
from evidently.metrics import DatasetCorrelationsMetric
from evidently.metrics import DatasetDriftMetric
from evidently.metrics import DataDriftTable
from evidently.metrics import EmbeddingsDriftMetric

from evidently.metrics.data_drift.embedding_drift_methods import model, distance, ratio, mmd

from evidently.descriptors import TextLength, TriggerWordsPresence, OOV, NonLetterCharacterPercentage
from evidently.descriptors import SentenceCount, WordCount, Sentiment, RegExp

In [ ]:
import nltk
nltk.download('words')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('vader_lexicon')

## Multimodal data with raw text

In [ ]:
reviews_data = datasets.fetch_openml(name='Womens-E-Commerce-Clothing-Reviews', version=2, as_frame='auto')
reviews = reviews_data.frame

In [ ]:
reviews['prediction'] = reviews['Rating']
reviews_ref = reviews[reviews.Rating > 3].sample(n=5000, replace=True, ignore_index=True, random_state=42)
reviews_cur = reviews[reviews.Rating < 3].sample(n=5000, replace=True, ignore_index=True, random_state=42)

In [ ]:
column_mapping = ColumnMapping(
    numerical_features=['Age', 'Positive_Feedback_Count'],
    categorical_features=['Division_Name', 'Department_Name', 'Class_Name'],
    text_features=['Review_Text', 'Title']
)

## Text data overview

In [ ]:
#NO descriptors

text_overview_report = Report(metrics=[
    TextOverviewPreset(column_name="Review_Text")
])

text_overview_report.run(reference_data=reviews_ref, current_data=reviews_cur, column_mapping=column_mapping)
text_overview_report.show(mode='inline')

In [ ]:
#WITH descriptors
text_overview_report = Report(metrics=[
    TextOverviewPreset(column_name="Review_Text", descriptors={
        "Review Text OOV" : OOV(),
        "Review Text Non Letter %" : NonLetterCharacterPercentage(),
        "Review Text Length" : TextLength(),
        "Reviews about Dress" : TriggerWordsPresence(words_list=['dress', 'gown']),
        "Review about Blouses" : TriggerWordsPresence(words_list=['blouse', 'shirt']),
        "Review Sentence Count" : SentenceCount(),
        "Review Word Count" : WordCount(),
        "Review Sentiment" : Sentiment(),
        "Review questions": RegExp(reg_exp=r'.*\?.*'),
    })
])

text_overview_report.run(reference_data=reviews_ref, current_data=reviews_cur, column_mapping=column_mapping)
text_overview_report.show(mode='inline')

## Text data drift with descriptors

In [ ]:
table_column_metrics_report = Report(metrics=[
    DatasetDriftMetric(columns=["Age", "Review_Text"]),
    DataDriftTable(columns=["Age", "Review_Text"]),

])

table_column_metrics_report.run(reference_data=reviews_ref, current_data=reviews_cur, column_mapping=column_mapping)
table_column_metrics_report.show(mode='inline')

In [ ]:
table_column_metrics_report = Report(metrics=[
    ColumnSummaryMetric(column_name = RegExp(reg_exp=r'.*\?.*', display_name="Questions").for_column("Review_Text")),
    ColumnDriftMetric(column_name = SentenceCount(display_name="SentenceCount").for_column("Review_Text")),
    ColumnCorrelationsMetric(column_name = WordCount(display_name="WordCount").for_column("Review_Text")),
    ColumnDistributionMetric(column_name = Sentiment(display_name="Sentiment").for_column("Review_Text")),
    ColumnValueRangeMetric(column_name = TextLength(display_name="TextLength").for_column("Review_Text"), left=0, right=20)
])

table_column_metrics_report.run(reference_data=reviews_ref, current_data=reviews_cur, column_mapping=column_mapping)
table_column_metrics_report.show(mode='inline')

## Embedding Drift Detection

In [ ]:

embeddings_data = datasets.fetch_lfw_people()
embeddings_data = pd.DataFrame(embeddings_data['data'])
embeddings_data.columns = ['col_' + str(x) for x in embeddings_data.columns]

embeddings_data = embeddings_data.iloc[:5100, :31]

embeddings_data_shifted = embeddings_data.copy()
embeddings_data_shifted.iloc[2500:5000, :5] = 0

In [ ]:
column_mapping = ColumnMapping(
    embeddings={'small_subset': embeddings_data.columns[:10], 'big_subset': embeddings_data.columns[10:29]},
    target=embeddings_data.columns[30]
)


In [ ]:
report = Report(metrics=[
    EmbeddingsDriftMetric('small_subset')
])

report.run(reference_data = embeddings_data[:2500], current_data = embeddings_data[2500:5000], 
           column_mapping = column_mapping)
report.show(mode='inline')

In [ ]:
report = Report(metrics = [
    EmbeddingsDriftMetric('small_subset', 
                          drift_method = ratio(
                              component_stattest = 'wasserstein',
                              component_stattest_threshold = 0.1,
                              threshold = 0.2,
                              pca_components = None,
                          )
                         )
])

report.run(reference_data = embeddings_data_shifted[:2500], current_data = embeddings_data_shifted[2500:5000],  
           column_mapping = column_mapping)
report.show(mode='inline')